In [235]:
from sklearn import preprocessing
from time import time
from sklearn.feature_extraction import DictVectorizer
import numpy as np
import csv
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from sklearn import metrics
from sklearn.cluster import KMeans
from sklearn.datasets import load_diabetes
from sklearn.decomposition import PCA
from sklearn.preprocessing import scale
from sklearn.feature_selection import VarianceThreshold
from sklearn.metrics import silhouette_samples, silhouette_score
from operator import truediv
from sklearn.metrics import pairwise_distances
import pandas as pd
get_ipython().magic('matplotlib')

df=pd.read_csv('C:/diabetic_data_processed_withweight.csv',';')
to_del = ['encounter_id', 'patient_nbr','medical_specialty','payer_code']
print (to_del)
#Filter_selected cols
filtered_cols = [c for c in df.columns if (c not in to_del) ]#and ('ENF' not in c)
df_2 = df[filtered_cols]
print ("df_2",df_2.shape)
print(df_2.columns)

Using matplotlib backend: Qt5Agg
['encounter_id', 'patient_nbr', 'medical_specialty', 'payer_code']
df_2 (101766, 46)
Index(['race', 'gender', 'age', 'weight', 'admission_type_id',
       'discharge_disposition_id', 'admission_source_id', 'time_in_hospital',
       'num_lab_procedures', 'num_procedures', 'num_medications',
       'number_outpatient', 'number_emergency', 'number_inpatient', 'diag_1',
       'diag_2', 'diag_3', 'number_diagnoses', 'max_glu_serum', 'A1Cresult',
       'metformin', 'repaglinide', 'nateglinide', 'chlorpropamide',
       'glimepiride', 'acetohexamide', 'glipizide', 'glyburide', 'tolbutamide',
       'pioglitazone', 'rosiglitazone', 'acarbose', 'miglitol', 'troglitazone',
       'tolazamide', 'examide', 'citoglipton', 'insulin',
       'glyburide-metformin', 'glipizide-metformin',
       'glimepiride-pioglitazone', 'metformin-rosiglitazone',
       'metformin-pioglitazone', 'change', 'diabetesMed', 'readmitted'],
      dtype='object')


In [256]:
for i in range (0,df_2['diag_3'].size):
    if ((df_2.loc[i,'diag_1']==410 or df_2.loc[i,'diag_1']==412) or (df_2.loc[i,'diag_2']==410 or df_2.loc[i,'diag_2']==412) or (df_2.loc[i,'diag_3']==410 or df_2.loc[i,'diag_3']==412)):
        #print(df_2['diag_1'],df_2['diag_2'],df_2['diag_3'])
        df_2.loc[i,'Myocardial_infarction']=1
    else:
        df_2.loc[i,'Myocardial_infarction']=0

In [218]:
print(df_2['Myocardial_infarction'].astype('category').value_counts())
listA=[]
listA1=[]
listA=df_2.loc[:,'Myocardial_infarction']==1
for x in range (0,len(listA)):
    if(listA[x]==True):
        listA1.append(x)
listC=[]
listC1=[]
listC=df_2.loc[:,'Myocardial_infarction']==1
for x in range (0,len(listC)):
    if(listC[x]==True):
        listC1.append(x)

print(df_2['A1Cresult'].describe())
mean=df_2['A1Cresult'].mean()
print("mean",mean)
listB1=[]
listB=df_2.loc[:,'A1Cresult']> mean
listD1=[]
listD=df_2.loc[:,'A1Cresult']< mean
listD.describe()
for x in range (0,len(listB)):
    if(listB[x]==True):
        listB1.append(x)
for x in range (0,len(listD)):
    if(listD[x]==True):
        listD1.append(x)
#No healty and bad social
a=len(set(listB1).intersection(listA1))
#Healthy and bad social
b=len(set(listB1).intersection(listC1))
#No healty and social
c=len(set(listD1).intersection(listA1))
#Healthy and social
d=len(set(listD1).intersection(listC1))

print(len(listB1), len(listA1), len(listC1),len(listD1))
print(a,b,c,d)

OR=(a*d)/(c*d)
RR=(a/(a+b)) / (c/(c+d))
RD=(a/(a+b)) - (c/(c+d))
print("Weight-->","RR: ",RR,", RD: ",RD,", OR: ",OR)
#Characteristics
listMI=[]
for x in range (0,len(df_2)):
    if (df_2.loc[x,'Myocardial_infarction']==1):
        listMI.append(df_2.loc[x,:])    
df_MI = pd.DataFrame(listMI)
df_MI['age'].describe()

import scipy as sp
import scipy.stats
def mean_confidence_interval(data, confidence=0.95):
    a = 1.0*np.array(data)
    n = len(a)
    m, se = np.mean(a), scipy.stats.sem(a)
    h = se * sp.stats.t._ppf((1+confidence)/2., n-1)
    return m, m-h, m+h
print("Age",mean_confidence_interval(df_MI['age']))
print("Gender",df_MI['gender'].astype('category').value_counts())
print("Race",df_MI['race'].astype('category').value_counts())
print("Readmitted",df_MI['readmitted'].astype('category').value_counts())

0.0    96985
1.0     4781
Name: Myocardial_infarction, dtype: int64
count    101766.000000
mean          0.242537
std           3.124083
min          -1.000000
25%          -1.000000
50%          -1.000000
75%          -1.000000
max           9.000000
Name: A1Cresult, dtype: float64
mean 0.2425368001100564
17018 4781 4781 84748
1028 1028 3753 3753
Weight--> RR:  1.0 , RD:  0.0 , OR:  0.2739142019717559
Age (74.080736247646939, 73.703162351840874, 74.458310143453005)
Gender 0.0    2645
1.0    2136
Name: gender, dtype: int64
Race 0.0    3937
1.0     533
2.0      98
3.0      68
4.0      24
Name: race, dtype: int64
Readmitted 0.0     2856
31.0    1432
29.0     493
Name: readmitted, dtype: int64


In [257]:
for i in range (0,df_2['diag_3'].size):
    if (df_2.loc[i,'diag_1']==428 or df_2.loc[i,'diag_2']==428 or (df_2.loc[i,'diag_3']==428)):
        #print(df_2['diag_1'],df_2['diag_2'],df_2['diag_3'])
        df_2.loc[i,'Congestive_heart_failure']=1
    else:
        df_2.loc[i,'Congestive_heart_failure']=0      

In [220]:
print(df_2['Congestive_heart_failure'].astype('category').value_counts())
listA=[]
listA1=[]
listA=df_2.loc[:,'Congestive_heart_failure']==1
for x in range (0,len(listA)):
    if(listA[x]==True):
        listA1.append(x)
listC=[]
listC1=[]
listC=df_2.loc[:,'Congestive_heart_failure']==1
for x in range (0,len(listC)):
    if(listC[x]==True):
        listC1.append(x)

print(df_2['weight'].describe())
mean=df_2['weight'].mean()
print("mean",mean)
listB1=[]
listB=df_2.loc[:,'weight']> mean
listD1=[]
listD=df_2.loc[:,'weight']< mean
listD.describe()
for x in range (0,len(listB)):
    if(listB[x]==True):
        listB1.append(x)
for x in range (0,len(listD)):
    if(listD[x]==True):
        listD1.append(x)
#No healty and bad social
a=len(set(listB1).intersection(listA1))
#Healthy and bad social
b=len(set(listB1).intersection(listC1))
#No healty and social
c=len(set(listD1).intersection(listA1))
#Healthy and social
d=len(set(listD1).intersection(listC1))

print(len(listB1), len(listA1), len(listC1),len(listD1))
print(a,b,c,d)

OR=(a*d)/(c*d)
RR=(a/(a+b)) / (c/(c+d))
RD=(a/(a+b)) - (c/(c+d))
print("Weight-->","RR: ",RR,", RD: ",RD,", OR: ",OR)

listA=[]
listA1=[]
listA=df_2.loc[:,'Congestive_heart_failure']==1
for x in range (0,len(listA)):
    if(listA[x]==True):
        listA1.append(x)
listC=[]
listC1=[]
listC=df_2.loc[:,'Congestive_heart_failure']==1
for x in range (0,len(listC)):
    if(listC[x]==True):
        listC1.append(x)

print(df_2['A1Cresult'].describe())
mean=df_2['A1Cresult'].mean()
print("mean",mean)
listB1=[]
listB=df_2.loc[:,'A1Cresult']> mean
listD1=[]
listD=df_2.loc[:,'A1Cresult']< mean
listD.describe()
for x in range (0,len(listB)):
    if(listB[x]==True):
        listB1.append(x)
for x in range (0,len(listD)):
    if(listD[x]==True):
        listD1.append(x)
#No healty and bad social
a=len(set(listB1).intersection(listA1))
#Healthy and bad social
b=len(set(listB1).intersection(listC1))
#No healty and social
c=len(set(listD1).intersection(listA1))
#Healthy and social
d=len(set(listD1).intersection(listC1))

print(len(listB1), len(listA1), len(listC1),len(listD1))
print(a,b,c,d)

OR=(a*d)/(c*d)
RR=(a/(a+b)) / (c/(c+d))
RD=(a/(a+b)) - (c/(c+d))
print("A1CResult-->","RR: ",RR,", RD: ",RD,", OR: ",OR)
#Characteristics
listCHF=[]
for x in range (0,len(df_2)):
    if (df_2.loc[x,'Congestive_heart_failure']==1):
        listCHF.append(df_2.loc[x,:])    
df_CHF = pd.DataFrame(listCHF)
df_CHF['age'].describe()

print("Age",mean_confidence_interval(df_CHF['age']))
print("Gender",df_CHF['gender'].astype('category').value_counts())
print("Race",df_CHF['race'].astype('category').value_counts())
print("Readmitted",df_CHF['readmitted'].astype('category').value_counts())

0    84302
1    17464
Name: Congestive_heart_failure, dtype: int64


C:\Users\laia.subirats\AppData\Local\Continuum\Anaconda3\lib\site-packages\numpy\lib\function_base.py:4116: RuntimeWarning: Invalid value encountered in percentile
  interpolation=interpolation)


count    3197.000000
mean       92.595246
std        25.465480
min        25.000000
25%              NaN
50%              NaN
75%              NaN
max       201.000000
Name: weight, dtype: float64
mean 92.59524554269628
819 17464 17464 2378
109 109 332 332
Weight--> RR:  1.0 , RD:  0.0 , OR:  0.32831325301204817
count    101766.000000
mean          0.242537
std           3.124083
min          -1.000000
25%          -1.000000
50%          -1.000000
75%          -1.000000
max           9.000000
Name: A1Cresult, dtype: float64
mean 0.2425368001100564
17018 17464 17464 84748
2712 2712 14752 14752
A1CResult--> RR:  1.0 , RD:  0.0 , OR:  0.18383947939262474
Age (77.187356848373796, 76.999992235787019, 77.374721460960572)
Gender 1.0    9547
0.0    7917
Name: gender, dtype: int64
Race 0.0    13273
1.0     3351
3.0      262
2.0      186
4.0       61
Name: race, dtype: int64
Readmitted 0.0     7809
31.0    7365
29.0    2290
Name: readmitted, dtype: int64


In [241]:
for i in range (0,df_2['diag_3'].size):
    if (df_2.loc[i,'diag_1'] in (443,441,785,43) 
        or df_2.loc[i,'diag_2'] in (443,441,785,43)  
        or df_2.loc[i,'diag_3'] in (443,441,785,43)):
        df_2.loc[i,'Peripheral_vascular_disease']=1
    else:
        df_2.loc[i,'Peripheral_vascular_disease']=0

C:\Users\laia.subirats\AppData\Local\Continuum\Anaconda3\lib\site-packages\pandas\core\indexing.py:465: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


In [244]:
print(df_2['Peripheral_vascular_disease'].astype('category').value_counts())
listA=[]
listA1=[]
listA=df_2.loc[:,'Peripheral_vascular_disease']==1
for x in range (0,len(listA)):
    if(listA[x]==True):
        listA1.append(x)
listC=[]
listC1=[]
listC=df_2.loc[:,'Peripheral_vascular_disease']==1
for x in range (0,len(listC)):
    if(listC[x]==True):
        listC1.append(x)

print(df_2['weight'].describe())
mean=df_2['weight'].mean()
print("mean",mean)
listB1=[]
listB=df_2.loc[:,'weight']> mean
listD1=[]
listD=df_2.loc[:,'weight']< mean
listD.describe()
for x in range (0,len(listB)):
    if(listB[x]==True):
        listB1.append(x)
for x in range (0,len(listD)):
    if(listD[x]==True):
        listD1.append(x)
#No healty and bad social
a=len(set(listB1).intersection(listA1))
#Healthy and bad social
b=len(set(listB1).intersection(listC1))
#No healty and social
c=len(set(listD1).intersection(listA1))
#Healthy and social
d=len(set(listD1).intersection(listC1))

print(len(listB1), len(listA1), len(listC1),len(listD1))
print(a,b,c,d)

OR=(a*d)/(c*d)
RR=(a/(a+b)) / (c/(c+d))
RD=(a/(a+b)) - (c/(c+d))
print("Weight-->","RR: ",RR,", RD: ",RD,", OR: ",OR)

listA=[]
listA1=[]
listA=df_2.loc[:,'Peripheral_vascular_disease']==1
for x in range (0,len(listA)):
    if(listA[x]==True):
        listA1.append(x)
listC=[]
listC1=[]
listC=df_2.loc[:,'Peripheral_vascular_disease']==1
for x in range (0,len(listC)):
    if(listC[x]==True):
        listC1.append(x)

print(df_2['A1Cresult'].describe())
mean=df_2['A1Cresult'].mean()
print("mean",mean)
listB1=[]
listB=df_2.loc[:,'A1Cresult']> mean
listD1=[]
listD=df_2.loc[:,'A1Cresult']< mean
listD.describe()
for x in range (0,len(listB)):
    if(listB[x]==True):
        listB1.append(x)
for x in range (0,len(listD)):
    if(listD[x]==True):
        listD1.append(x)
#No healty and bad social
a=len(set(listB1).intersection(listA1))
#Healthy and bad social
b=len(set(listB1).intersection(listC1))
#No healty and social
c=len(set(listD1).intersection(listA1))
#Healthy and social
d=len(set(listD1).intersection(listC1))

print(len(listB1), len(listA1), len(listC1),len(listD1))
print(a,b,c,d)

OR=(a*d)/(c*d)
RR=(a/(a+b)) / (c/(c+d))
RD=(a/(a+b)) - (c/(c+d))
print("A1CResult-->","RR: ",RR,", RD: ",RD,", OR: ",OR)
#Characteristics
listCHF=[]
for x in range (0,len(df_2)):
    if (df_2.loc[x,'Peripheral_vascular_disease']==1):
        listCHF.append(df_2.loc[x,:])    
df_CHF = pd.DataFrame(listCHF)
print(df_CHF['age'].describe())

print("Age",mean_confidence_interval(df_CHF['age']))
print("Gender",df_CHF['gender'].astype('category').value_counts())
print("Race",df_CHF['race'].astype('category').value_counts())
print("Readmitted",df_CHF['readmitted'].astype('category').value_counts())

0    99851
1     1915
Name: Peripheral_vascular_disease, dtype: int64


C:\Users\laia.subirats\AppData\Local\Continuum\Anaconda3\lib\site-packages\numpy\lib\function_base.py:4116: RuntimeWarning: Invalid value encountered in percentile
  interpolation=interpolation)


count    3197.000000
mean       92.595246
std        25.465480
min        25.000000
25%              NaN
50%              NaN
75%              NaN
max       201.000000
Name: weight, dtype: float64
mean 92.59524554269628
819 1915 1915 2378
23 23 41 41
Weight--> RR:  1.0 , RD:  0.0 , OR:  0.5609756097560976
count    101766.000000
mean          0.242537
std           3.124083
min          -1.000000
25%          -1.000000
50%          -1.000000
75%          -1.000000
max           9.000000
Name: A1Cresult, dtype: float64
mean 0.2425368001100564
17018 1915 1915 84748
281 281 1634 1634
A1CResult--> RR:  1.0 , RD:  0.0 , OR:  0.17197062423500611
Age (72.44908616187989, 71.828490629770698, 73.069681693989082)
Gender 0.0    1005
1.0     910
Name: gender, dtype: int64
Race 0.0    1478
1.0     318
3.0      36
2.0      29
4.0       7
Name: race, dtype: int64
Readmitted 0.0     1040
31.0     619
29.0     256
Name: readmitted, dtype: int64


In [246]:
for i in range (0,df_2['diag_3'].size):
    if ((df_2.loc[i,'diag_1']>=430 and df_2.loc[i,'diag_1']<=438) or (df_2.loc[i,'diag_2']>=430 and df_2.loc[i,'diag_2']<=438) or (df_2.loc[i,'diag_3']>=430 and df_2.loc[i,'diag_3']<=438)):
        df_2.loc[i,'Cerebrovascular_disease']=1
    else:
        df_2.loc[i,'Cerebrovascular_disease']=0

C:\Users\laia.subirats\AppData\Local\Continuum\Anaconda3\lib\site-packages\pandas\core\indexing.py:288: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[key] = _infer_fill_value(value)
C:\Users\laia.subirats\AppData\Local\Continuum\Anaconda3\lib\site-packages\pandas\core\indexing.py:465: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


In [247]:
print(df_2['Cerebrovascular_disease'].astype('category').value_counts())
listA=[]
listA1=[]
listA=df_2.loc[:,'Cerebrovascular_disease']==1
for x in range (0,len(listA)):
    if(listA[x]==True):
        listA1.append(x)
listC=[]
listC1=[]
listC=df_2.loc[:,'Cerebrovascular_disease']==1
for x in range (0,len(listC)):
    if(listC[x]==True):
        listC1.append(x)

print(df_2['weight'].describe())
mean=df_2['weight'].mean()
print("mean",mean)
listB1=[]
listB=df_2.loc[:,'weight']> mean
listD1=[]
listD=df_2.loc[:,'weight']< mean
listD.describe()
for x in range (0,len(listB)):
    if(listB[x]==True):
        listB1.append(x)
for x in range (0,len(listD)):
    if(listD[x]==True):
        listD1.append(x)
#No healty and bad social
a=len(set(listB1).intersection(listA1))
#Healthy and bad social
b=len(set(listB1).intersection(listC1))
#No healty and social
c=len(set(listD1).intersection(listA1))
#Healthy and social
d=len(set(listD1).intersection(listC1))

print(len(listB1), len(listA1), len(listC1),len(listD1))
print(a,b,c,d)

OR=(a*d)/(c*d)
RR=(a/(a+b)) / (c/(c+d))
RD=(a/(a+b)) - (c/(c+d))
print("Weight-->","RR: ",RR,", RD: ",RD,", OR: ",OR)

listA=[]
listA1=[]
listA=df_2.loc[:,'Cerebrovascular_disease']==1
for x in range (0,len(listA)):
    if(listA[x]==True):
        listA1.append(x)
listC=[]
listC1=[]
listC=df_2.loc[:,'Cerebrovascular_disease']==1
for x in range (0,len(listC)):
    if(listC[x]==True):
        listC1.append(x)

print(df_2['A1Cresult'].describe())
mean=df_2['A1Cresult'].mean()
print("mean",mean)
listB1=[]
listB=df_2.loc[:,'A1Cresult']> mean
listD1=[]
listD=df_2.loc[:,'A1Cresult']< mean
listD.describe()
for x in range (0,len(listB)):
    if(listB[x]==True):
        listB1.append(x)
for x in range (0,len(listD)):
    if(listD[x]==True):
        listD1.append(x)
#No healty and bad social
a=len(set(listB1).intersection(listA1))
#Healthy and bad social
b=len(set(listB1).intersection(listC1))
#No healty and social
c=len(set(listD1).intersection(listA1))
#Healthy and social
d=len(set(listD1).intersection(listC1))

print(len(listB1), len(listA1), len(listC1),len(listD1))
print(a,b,c,d)

OR=(a*d)/(c*d)
RR=(a/(a+b)) / (c/(c+d))
RD=(a/(a+b)) - (c/(c+d))
print("A1CResult-->","RR: ",RR,", RD: ",RD,", OR: ",OR)
#Characteristics
listCHF=[]
for x in range (0,len(df_2)):
    if (df_2.loc[x,'Cerebrovascular_disease']==1):
        listCHF.append(df_2.loc[x,:])    
df_CHF = pd.DataFrame(listCHF)
print(df_CHF['age'].describe())

print("Age",mean_confidence_interval(df_CHF['age']))
print("Gender",df_CHF['gender'].astype('category').value_counts())
print("Race",df_CHF['race'].astype('category').value_counts())
print("Readmitted",df_CHF['readmitted'].astype('category').value_counts())

0.0    96197
1.0     5569
Name: Cerebrovascular_disease, dtype: int64


C:\Users\laia.subirats\AppData\Local\Continuum\Anaconda3\lib\site-packages\numpy\lib\function_base.py:4116: RuntimeWarning: Invalid value encountered in percentile
  interpolation=interpolation)


count    3197.000000
mean       92.595246
std        25.465480
min        25.000000
25%              NaN
50%              NaN
75%              NaN
max       201.000000
Name: weight, dtype: float64
mean 92.59524554269628
819 5569 5569 2378
27 27 130 130
Weight--> RR:  1.0 , RD:  0.0 , OR:  0.2076923076923077
count    101766.000000
mean          0.242537
std           3.124083
min          -1.000000
25%          -1.000000
50%          -1.000000
75%          -1.000000
max           9.000000
Name: A1Cresult, dtype: float64
mean 0.2425368001100564
17018 5569 5569 84748
1132 1132 4437 4437
A1CResult--> RR:  1.0 , RD:  0.0 , OR:  0.2551273382916385
count    5569.000000
mean       75.760460
std        12.326413
min        10.000000
25%        70.000000
50%        80.000000
75%        80.000000
max       100.000000
Name: age, dtype: float64
Age (75.760459687556107, 75.436649424379453, 76.084269950732761)
Gender 1.0    2961
0.0    2608
Name: gender, dtype: int64
Race 0.0    3971
1.0    1243
3.0 

In [ ]:
for i in range (0,df_2['diag_3'].size):
    if (df_2.loc[i,'diag_1']==290 or df_2.loc[i,'diag_2']==290 or df_2.loc[i,'diag_3']==290):
        df_2.loc[i,'Dementia']=1
    else:
        df_2.loc[i,'Dementia']=0

C:\Users\laia.subirats\AppData\Local\Continuum\Anaconda3\lib\site-packages\pandas\core\indexing.py:288: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[key] = _infer_fill_value(value)
C:\Users\laia.subirats\AppData\Local\Continuum\Anaconda3\lib\site-packages\pandas\core\indexing.py:465: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


In [252]:
print(df_2['Dementia'].astype('category').value_counts())
listA=[]
listA1=[]
listA=df_2.loc[:,'Dementia']==1
for x in range (0,len(listA)):
    if(listA[x]==True):
        listA1.append(x)
listC=[]
listC1=[]
listC=df_2.loc[:,'Dementia']==1
for x in range (0,len(listC)):
    if(listC[x]==True):
        listC1.append(x)

print(df_2['weight'].describe())
mean=df_2['weight'].mean()
print("mean",mean)
listB1=[]
listB=df_2.loc[:,'weight']> mean
listD1=[]
listD=df_2.loc[:,'weight']< mean
listD.describe()
for x in range (0,len(listB)):
    if(listB[x]==True):
        listB1.append(x)
for x in range (0,len(listD)):
    if(listD[x]==True):
        listD1.append(x)
#No healty and bad social
a=len(set(listB1).intersection(listA1))
#Healthy and bad social
b=len(set(listB1).intersection(listC1))
#No healty and social
c=len(set(listD1).intersection(listA1))
#Healthy and social
d=len(set(listD1).intersection(listC1))

print(len(listB1), len(listA1), len(listC1),len(listD1))
print(a,b,c,d)

OR=(a*d)/(c*d)
#RR=(a/(a+b)) / (c/(c+d))
#RD=(a/(a+b)) - (c/(c+d))
print("Weight-->","OR: ",OR)

listA=[]
listA1=[]
listA=df_2.loc[:,'Dementia']==1
for x in range (0,len(listA)):
    if(listA[x]==True):
        listA1.append(x)
listC=[]
listC1=[]
listC=df_2.loc[:,'Dementia']==1
for x in range (0,len(listC)):
    if(listC[x]==True):
        listC1.append(x)

print(df_2['A1Cresult'].describe())
mean=df_2['A1Cresult'].mean()
print("mean",mean)
listB1=[]
listB=df_2.loc[:,'A1Cresult']> mean
listD1=[]
listD=df_2.loc[:,'A1Cresult']< mean
listD.describe()
for x in range (0,len(listB)):
    if(listB[x]==True):
        listB1.append(x)
for x in range (0,len(listD)):
    if(listD[x]==True):
        listD1.append(x)
#No healty and bad social
a=len(set(listB1).intersection(listA1))
#Healthy and bad social
b=len(set(listB1).intersection(listC1))
#No healty and social
c=len(set(listD1).intersection(listA1))
#Healthy and social
d=len(set(listD1).intersection(listC1))

print(len(listB1), len(listA1), len(listC1),len(listD1))
print(a,b,c,d)

OR=(a*d)/(c*d)
RR=(a/(a+b)) / (c/(c+d))
RD=(a/(a+b)) - (c/(c+d))
print("A1CResult-->","RR: ",RR,", RD: ",RD,", OR: ",OR)
#Characteristics
listCHF=[]
for x in range (0,len(df_2)):
    if (df_2.loc[x,'Dementia']==1):
        listCHF.append(df_2.loc[x,:])    
df_CHF = pd.DataFrame(listCHF)
print(df_CHF['age'].describe())

print("Age",mean_confidence_interval(df_CHF['age']))
print("Gender",df_CHF['gender'].astype('category').value_counts())
print("Race",df_CHF['race'].astype('category').value_counts())
print("Readmitted",df_CHF['readmitted'].astype('category').value_counts())

0.0    101555
1.0       211
Name: Dementia, dtype: int64


C:\Users\laia.subirats\AppData\Local\Continuum\Anaconda3\lib\site-packages\numpy\lib\function_base.py:4116: RuntimeWarning: Invalid value encountered in percentile
  interpolation=interpolation)


count    3197.000000
mean       92.595246
std        25.465480
min        25.000000
25%              NaN
50%              NaN
75%              NaN
max       201.000000
Name: weight, dtype: float64
mean 92.59524554269628
819 211 211 2378
0 0 4 4
Weight--> OR:  0.0
count    101766.000000
mean          0.242537
std           3.124083
min          -1.000000
25%          -1.000000
50%          -1.000000
75%          -1.000000
max           9.000000
Name: A1Cresult, dtype: float64
mean 0.2425368001100564
17018 211 211 84748
44 44 167 167
A1CResult--> RR:  1.0 , RD:  0.0 , OR:  0.2634730538922156
count    211.000000
mean      83.459716
std        9.898150
min       50.000000
25%       80.000000
50%       90.000000
75%       90.000000
max      100.000000
Name: age, dtype: float64
Age (83.459715639810426, 82.116421937900355, 84.803009341720497)
Gender 1.0    122
0.0     89
Name: gender, dtype: int64
Race 0.0    159
1.0     41
2.0      5
3.0      2
4.0      1
Name: race, dtype: int64
Readmitted 

In [253]:
for i in range (0,df_2['diag_3'].size):
    if ((df_2.loc[i,'diag_1']>=490 and df_2.loc[i,'diag_1']<=506) or (df_2.loc[i,'diag_2']>=490 and df_2.loc[i,'diag_2']<=506) or (df_2.loc[i,'diag_3']>=490 and df_2.loc[i,'diag_3']<=506)):
        df_2.loc[i,'Chronic_pulmonary_disease']=1
    else:
        df_2.loc[i,'Chronic_pulmonary_disease']=0

In [254]:
print(df_2['Chronic_pulmonary_disease'].astype('category').value_counts())
listA=[]
listA1=[]
listA=df_2.loc[:,'Chronic_pulmonary_disease']==1
for x in range (0,len(listA)):
    if(listA[x]==True):
        listA1.append(x)
listC=[]
listC1=[]
listC=df_2.loc[:,'Chronic_pulmonary_disease']==1
for x in range (0,len(listC)):
    if(listC[x]==True):
        listC1.append(x)

print(df_2['weight'].describe())
mean=df_2['weight'].mean()
print("mean",mean)
listB1=[]
listB=df_2.loc[:,'weight']> mean
listD1=[]
listD=df_2.loc[:,'weight']< mean
listD.describe()
for x in range (0,len(listB)):
    if(listB[x]==True):
        listB1.append(x)
for x in range (0,len(listD)):
    if(listD[x]==True):
        listD1.append(x)
#No healty and bad social
a=len(set(listB1).intersection(listA1))
#Healthy and bad social
b=len(set(listB1).intersection(listC1))
#No healty and social
c=len(set(listD1).intersection(listA1))
#Healthy and social
d=len(set(listD1).intersection(listC1))

print(len(listB1), len(listA1), len(listC1),len(listD1))
print(a,b,c,d)

OR=(a*d)/(c*d)
RR=(a/(a+b)) / (c/(c+d))
RD=(a/(a+b)) - (c/(c+d))
print("Weight-->","RR: ",RR,", RD: ",RD,", OR: ",OR)

listA=[]
listA1=[]
listA=df_2.loc[:,'Chronic_pulmonary_disease']==1
for x in range (0,len(listA)):
    if(listA[x]==True):
        listA1.append(x)
listC=[]
listC1=[]
listC=df_2.loc[:,'Chronic_pulmonary_disease']==1
for x in range (0,len(listC)):
    if(listC[x]==True):
        listC1.append(x)

print(df_2['A1Cresult'].describe())
mean=df_2['A1Cresult'].mean()
print("mean",mean)
listB1=[]
listB=df_2.loc[:,'A1Cresult']> mean
listD1=[]
listD=df_2.loc[:,'A1Cresult']< mean
listD.describe()
for x in range (0,len(listB)):
    if(listB[x]==True):
        listB1.append(x)
for x in range (0,len(listD)):
    if(listD[x]==True):
        listD1.append(x)
#No healty and bad social
a=len(set(listB1).intersection(listA1))
#Healthy and bad social
b=len(set(listB1).intersection(listC1))
#No healty and social
c=len(set(listD1).intersection(listA1))
#Healthy and social
d=len(set(listD1).intersection(listC1))

print(len(listB1), len(listA1), len(listC1),len(listD1))
print(a,b,c,d)

OR=(a*d)/(c*d)
RR=(a/(a+b)) / (c/(c+d))
RD=(a/(a+b)) - (c/(c+d))
print("A1CResult-->","RR: ",RR,", RD: ",RD,", OR: ",OR)
#Characteristics
listCHF=[]
for x in range (0,len(df_2)):
    if (df_2.loc[x,'Chronic_pulmonary_disease']==1):
        listCHF.append(df_2.loc[x,:])    
df_CHF = pd.DataFrame(listCHF)
print(df_CHF['age'].describe())

print("Age",mean_confidence_interval(df_CHF['age']))
print("Gender",df_CHF['gender'].astype('category').value_counts())
print("Race",df_CHF['race'].astype('category').value_counts())
print("Readmitted",df_CHF['readmitted'].astype('category').value_counts())

0.0    88126
1.0    13640
Name: Chronic_pulmonary_disease, dtype: int64


C:\Users\laia.subirats\AppData\Local\Continuum\Anaconda3\lib\site-packages\numpy\lib\function_base.py:4116: RuntimeWarning: Invalid value encountered in percentile
  interpolation=interpolation)


count    3197.000000
mean       92.595246
std        25.465480
min        25.000000
25%              NaN
50%              NaN
75%              NaN
max       201.000000
Name: weight, dtype: float64
mean 92.59524554269628
819 13640 13640 2378
92 92 233 233
Weight--> RR:  1.0 , RD:  0.0 , OR:  0.3948497854077253
count    101766.000000
mean          0.242537
std           3.124083
min          -1.000000
25%          -1.000000
50%          -1.000000
75%          -1.000000
max           9.000000
Name: A1Cresult, dtype: float64
mean 0.2425368001100564
17018 13640 13640 84748
2238 2238 11402 11402
A1CResult--> RR:  1.0 , RD:  0.0 , OR:  0.196281354148395
count    13640.000000
mean        73.488270
std         13.325404
min         10.000000
25%         70.000000
50%         70.000000
75%         80.000000
max        100.000000
Name: age, dtype: float64
Age (73.488269794721404, 73.264624460374989, 73.711915129067819)
Gender 1.0    7518
0.0    6122
Name: gender, dtype: int64
Race 0.0    11090
1.

In [258]:
print(df_2.columns)

Index(['race', 'gender', 'age', 'weight', 'admission_type_id',
       'discharge_disposition_id', 'admission_source_id', 'time_in_hospital',
       'num_lab_procedures', 'num_procedures', 'num_medications',
       'number_outpatient', 'number_emergency', 'number_inpatient', 'diag_1',
       'diag_2', 'diag_3', 'number_diagnoses', 'max_glu_serum', 'A1Cresult',
       'metformin', 'repaglinide', 'nateglinide', 'chlorpropamide',
       'glimepiride', 'acetohexamide', 'glipizide', 'glyburide', 'tolbutamide',
       'pioglitazone', 'rosiglitazone', 'acarbose', 'miglitol', 'troglitazone',
       'tolazamide', 'examide', 'citoglipton', 'insulin',
       'glyburide-metformin', 'glipizide-metformin',
       'glimepiride-pioglitazone', 'metformin-rosiglitazone',
       'metformin-pioglitazone', 'change', 'diabetesMed', 'readmitted',
       'Peripheral_vascular_disease', 'Cerebrovascular_disease', 'Dementia',
       'Chronic_pulmonary_disease', 'Myocardial_infarction',
       'Congestive_heart_fa